In [6]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import cross_validate, cross_val_predict, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [7]:
#!pip install db-dtypes
#!pip install --upgrade google-cloud-bigquery google-cloud-storage google-cloud-bigquery-storage

In [8]:
import google
print(google.__file__)

None


In [9]:
# Cloud Storage
from google.cloud import storage
storage_client = storage.Client(project='film-wizard-453315')

# BigQuery
from google.cloud import bigquery
from google.cloud import bigquery_storage
print("BigQuery Storage API is installed successfully!")
bigquery_client = bigquery.Client(project='film-wizard-453315')

print("Clients initialized successfully!")

BigQuery Storage API is installed successfully!
Clients initialized successfully!


## Import Scripts

In [688]:
client = bigquery_client

query = 'SELECT * FROM `film-wizard-453315.scripts_preproc.processed_scripts_full`'
script_df = client.query(query).to_dataframe()

In [689]:
import pandas as pd

In [690]:
# project_id = 'film-wizard-453315'  # GCP Project ID
# dataset_id = "scripts_preproc"  # dataset name
# table_id = 'processed_scripts_full'  # Replace with your table name
# client = bigquery_client
# script_df.to_gbq(f"{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

In [730]:
script_df = script_df.drop_duplicates(subset=['URL']).reset_index(drop= True)
script_df.dtypes, script_df.isnull().sum()

(lower_title           object
 Title                 object
 URL                   object
 Script                object
 Clean_Script          object
 joy                  float64
 neutral              float64
 sadness              float64
 surprise             float64
 anger                float64
 disgust              float64
 fear                 float64
 sentiment_entropy    float64
 sentiment_std        float64
 hapax                  Int64
 fkgl                 float64
 fre                  float64
 smog                 float64
 fog                  float64
 word count             Int64
 TTR                  float64
 script_length          Int64
 mean_word_length     float64
 dtype: object,
 lower_title             0
 Title                   0
 URL                     0
 Script                  0
 Clean_Script            0
 joy                   927
 neutral               466
 sadness               227
 surprise              262
 anger                1152
 disgust              72

In [692]:
query = '''
SELECT title, overview, release_date, runtime, genre_1, genre_2, genre_3

FROM `film-wizard-453315.tmdb_metadata.movie_details`
'''

# Run the query and store the results in a DataFrame
tmdb_df = client.query(query).to_dataframe()

#preprocess
tmdb_df['release_year'] = tmdb_df['release_date'].astype(str).str[:4]
tmdb_df['lower_title'] = tmdb_df['title'].apply(str.lower) + ' (' + tmdb_df['release_year'] + ')'

unique_genres = tmdb_df['genre_1'].unique().tolist()

In [693]:
len(unique_genres)

20

## Import TMDB


In [731]:
query = '''
SELECT tmdbId, title,  release_date, runtime, genre_1, genre_2, genre_3


FROM `film-wizard-453315.tmdb_metadata.movie_details`
'''

# Run the query and store the results in a DataFrame
tmdb_df = client.query(query).to_dataframe()

#preprocess
tmdb_df['release_year'] = tmdb_df['release_date'].astype(str).str[:4]
tmdb_df['lower_title'] = tmdb_df['title'].apply(str.lower) + ' (' + tmdb_df['release_year'] + ')'
tmdb_df['genres'] = tmdb_df['genre_1'].astype(str) + ' ' + tmdb_df['genre_2'].astype(str) + ' ' + tmdb_df['genre_3'].astype(str)
tmdb_df['genres'] = tmdb_df['genres'].str.replace('None', '').str.strip().apply(str.upper)
tmdb_df.drop(columns = ['genre_1','genre_2','genre_3','release_date'], inplace=True)
tmdb_df = tmdb_df.drop_duplicates(subset=['lower_title']).reset_index(drop= True)
# replace zero runtime with average
mean_runtime = tmdb_df[tmdb_df['runtime'] != 0]['runtime'].mean()
# Replace zeros with the calculated mean
tmdb_df['runtime'] = tmdb_df['runtime'].replace(0, mean_runtime)


tmdb_df.describe()


,tmdbId,runtime
count,86143.0,86137.000000
mean,239837.874453,92.279317
std,246666.044615,52.689389
min,2.0,1.000000
25%,46386.5,84.000000
50%,136966.0,93.000000
75%,379307.0,105.000000
max,1186337.0,12480.000000


In [797]:
#merge dataframes 
df_merged = pd.merge(script_df, tmdb_df, how = 'inner', on = 'lower_title')
#df = df.drop_duplicates(subset=['lower_title'])
df_merged['words_per_minute'] = df_merged['script_length'] / df_merged['runtime']
df_merged.columns

,lower_title,Title,URL,Script,Clean_Script,joy,neutral,sadness,surprise,anger,...,word count,TTR,script_length,mean_word_length,tmdbId,title,runtime,release_year,genres,words_per_minute
0,sombre (1998),Sombre (1998),/movie_script.php?movie=sombre,Thump him!\n Thump him!\n Behind...\n Behind y...,"[thump, him, thump, him, behind, behind, you, ...",0.200000,NaN,0.266667,0.066667,0.200000,...,483,0.308232,1567,3.729419,56653,Sombre,112.0,1998,DRAMA HORROR,13.991071
1,savior (1998),Savior (1998),/movie_script.php?movie=savior,"Christian, how are you, son?\n Good. You misse...","[christian, how, are, you, son, good, you, mis...",NaN,0.080000,0.280000,0.080000,0.240000,...,653,0.247161,2642,3.836866,2042,Savior,103.0,1998,HISTORY DRAMA WAR,25.650485
2,"aloha, bobby and rose (1975)","Aloha, Bobby and Rose (1975)",/movie_script.php?movie=aloha-bobby-and-rose,"From the big\n band years of the '40s,\n Artie...","[from, the, big, band, years, of, the, 40s, ar...",0.181818,0.254545,0.036364,0.236364,0.090909,...,969,0.160643,6032,3.730438,47178,"Aloha, Bobby and Rose",88.0,1975,DRAMA ROMANCE ACTION,68.545455
3,money train (1995),Money Train (1995),/movie_script.php?movie=money-train,MONEY TRAIN\n Merry Christmas.\n Watch it!\n T...,"[money, train, merry, christmas, watch, it, th...",0.074074,0.148148,0.092593,0.055556,0.277778,...,1104,0.199027,5547,3.843880,11517,Money Train,110.0,1995,ACTION COMEDY CRIME,50.427273
4,blue story (2019),Blue Story (2019),/movie_script.php?movie=blue-story,1\n [RADIO CUTTING IN AND OUT]\n [MALE REPORTE...,"[radio, cutting, in, and, out, male, reporter,...",0.083333,0.159091,0.181818,0.075758,0.371212,...,1764,0.130880,13478,3.942573,621191,Blue Story,91.0,2019,CRIME DRAMA ROMANCE,148.10989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19619,marius (1931),Marius (1931),/movie_script.php?movie=marius,"- Hey, Marius.\n - Yes, Fanny?\n - What are yo...","[hey, marius, yes, fanny, what, are, you, thin...",0.076923,0.142857,0.252747,0.142857,0.131868,...,1601,0.165993,9645,3.903577,42818,Marius,127.0,1931,COMEDY DRAMA ROMANCE,75.944882
19620,my brother the devil (2012),My Brother the Devil (2012),/movie_script.php?movie=my-brother-the-devil,Brother!\n What is it? Ah...\n - All fucked up...,"[brother, what, is, it, ah, all, fucked, up, m...",0.042553,0.106383,0.106383,0.234043,0.212766,...,848,0.169702,4997,3.768261,84316,My Brother the Devil,111.0,2012,DRAMA,45.018018
19621,hellion (2014),Hellion (2014),/movie_script.php?movie=hellion,Jacob! Wait!\n Check this out guys!\n Hey! Com...,"[jacob, wait, check, this, out, guys, hey, com...",0.128205,0.102564,0.230769,0.153846,0.230769,...,901,0.218902,4116,3.849854,244536,Hellion,98.0,2014,THRILLER DRAMA,42.0
19622,cross wars (2017),Cross Wars (2017),/movie_script.php?movie=cross-wars,"Hi, you don't happen to have the...\n The cros...","[hi, you, dont, happen, to, have, the, the, cr...",0.093333,0.360000,0.146667,0.093333,0.040000,...,1392,0.180031,7732,3.928220,412452,Cross Wars,98.0,2017,ACTION,78.897959


In [902]:
cols = ['fear', 'sadness', 'joy', 'neutral', 'anger', 'surprise', 'disgust']
df_merged.loc[:, cols] = df_merged[cols].fillna(0)
df_merged.loc[df_merged['fre'] < 0, 'fre'] = 0
df_merged.loc[df_merged['fkgl'] <= 0, 'fkgl'] = 0.0001
df_merged[['sentiment_entropy', 'sentiment_std']] = df[['sentiment_entropy', 'sentiment_std']].fillna(df[['sentiment_entropy', 'sentiment_std']].mean())
df_merged = df_merged.dropna()
df_merged = df_merged.drop_duplicates(subset=['lower_title'])
df_merged.isnull().sum()
#df_merged= df_merged.set_index('tmdbId', drop = True)
df_merged


,lower_title,Title,URL,Script,Clean_Script,joy,neutral,sadness,surprise,anger,...,TTR,script_length,mean_word_length,tmdbId,title,runtime,release_year,genres,words_per_minute,clean_genres
0,sombre (1998),Sombre (1998),/movie_script.php?movie=sombre,Thump him!\n Thump him!\n Behind...\n Behind y...,"[thump, him, thump, him, behind, behind, you, ...",0.200000,0.000000,0.266667,0.066667,0.200000,...,0.308232,1567,3.729419,56653,Sombre,112.0,1998,DRAMA HORROR,13.991071,drama horror
1,savior (1998),Savior (1998),/movie_script.php?movie=savior,"Christian, how are you, son?\n Good. You misse...","[christian, how, are, you, son, good, you, mis...",0.000000,0.080000,0.280000,0.080000,0.240000,...,0.247161,2642,3.836866,2042,Savior,103.0,1998,HISTORY DRAMA WAR,25.650485,history drama war
2,"aloha, bobby and rose (1975)","Aloha, Bobby and Rose (1975)",/movie_script.php?movie=aloha-bobby-and-rose,"From the big\n band years of the '40s,\n Artie...","[from, the, big, band, years, of, the, 40s, ar...",0.181818,0.254545,0.036364,0.236364,0.090909,...,0.160643,6032,3.730438,47178,"Aloha, Bobby and Rose",88.0,1975,DRAMA ROMANCE ACTION,68.545455,drama romance action
3,money train (1995),Money Train (1995),/movie_script.php?movie=money-train,MONEY TRAIN\n Merry Christmas.\n Watch it!\n T...,"[money, train, merry, christmas, watch, it, th...",0.074074,0.148148,0.092593,0.055556,0.277778,...,0.199027,5547,3.843880,11517,Money Train,110.0,1995,ACTION COMEDY CRIME,50.427273,action comedy crime
4,blue story (2019),Blue Story (2019),/movie_script.php?movie=blue-story,1\n [RADIO CUTTING IN AND OUT]\n [MALE REPORTE...,"[radio, cutting, in, and, out, male, reporter,...",0.083333,0.159091,0.181818,0.075758,0.371212,...,0.130880,13478,3.942573,621191,Blue Story,91.0,2019,CRIME DRAMA ROMANCE,148.10989,crime drama romance
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19619,marius (1931),Marius (1931),/movie_script.php?movie=marius,"- Hey, Marius.\n - Yes, Fanny?\n - What are yo...","[hey, marius, yes, fanny, what, are, you, thin...",0.076923,0.142857,0.252747,0.142857,0.131868,...,0.165993,9645,3.903577,42818,Marius,127.0,1931,COMEDY DRAMA ROMANCE,75.944882,comedy drama romance
19620,my brother the devil (2012),My Brother the Devil (2012),/movie_script.php?movie=my-brother-the-devil,Brother!\n What is it? Ah...\n - All fucked up...,"[brother, what, is, it, ah, all, fucked, up, m...",0.042553,0.106383,0.106383,0.234043,0.212766,...,0.169702,4997,3.768261,84316,My Brother the Devil,111.0,2012,DRAMA,45.018018,drama
19621,hellion (2014),Hellion (2014),/movie_script.php?movie=hellion,Jacob! Wait!\n Check this out guys!\n Hey! Com...,"[jacob, wait, check, this, out, guys, hey, com...",0.128205,0.102564,0.230769,0.153846,0.230769,...,0.218902,4116,3.849854,244536,Hellion,98.0,2014,THRILLER DRAMA,42.0,thriller drama
19622,cross wars (2017),Cross Wars (2017),/movie_script.php?movie=cross-wars,"Hi, you don't happen to have the...\n The cros...","[hi, you, dont, happen, to, have, the, the, cr...",0.093333,0.360000,0.146667,0.093333,0.040000,...,0.180031,7732,3.928220,412452,Cross Wars,98.0,2017,ACTION,78.897959,action


In [947]:
# import pandas as pd
# import string
# import nltk
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import CountVectorizer
# import re  # Import regular expression module

# # Download necessary NLTK resources
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# # Initialize Stopwords, Lemmatizer
# stop_words = set(stopwords.words('english'))
# lemmatizer = WordNetLemmatizer()

# # Preprocessing Function
# def preprocess_text(text):
#     if not isinstance(text, str):  # Handle non-string values
#         return ''
#     # Remove digits
#     text = re.sub(r'\d+', '', text)
#     # Tokenize & Lowercase
#     tokens = word_tokenize(text.lower())
#     # Remove punctuation & stopwords
#     tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
#     # Lemmatize words
#     tokens = [lemmatizer.lemmatize(word) for word in tokens]
#     return ' '.join(tokens)

# # Apply preprocessing
# df["clean_overview"] = df["overview"].apply(preprocess_text)

# # Vectorize using CountVectorizer (Bag-of-Words Model)
# count_vectorizer = CountVectorizer()
# X_bow = count_vectorizer.fit_transform(df["clean_overview"])
# vocab = count_vectorizer.get_feature_names_out()

# # Compute word frequencies
# word_counts = X_bow.toarray().sum(axis=0)  # Sum word occurrences across all rows

# # Identify words appearing more than 10 times
# words_to_keep = [word for word, count in zip(vocab, word_counts) if count > 100]

# # Refit CountVectorizer with filtered vocabulary
# filtered_vectorizer = CountVectorizer(vocabulary=words_to_keep)
# X_bow_filtered = filtered_vectorizer.fit_transform(df["clean_overview"])

# # Convert to DataFrame and join with the original dataframe
# vectorized_texts = pd.DataFrame(
#     X_bow_filtered.toarray(),
#     columns=filtered_vectorizer.get_feature_names_out(),
#     index=df.index
# )
# vectorized_texts.head()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tigranarzumanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tigranarzumanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tigranarzumanov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


KeyError: 'overview'

In [903]:
from sklearn.feature_extraction.text import TfidfVectorizer
df1 = df_merged
def preprocess_genres(genres):
    # Tokenize the genres (in this case, it's a simple space-separated string)
    genres = genres.lower()  # Convert to lowercase
    genres = genres.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return ' '.join(word_tokenize(genres))  # Tokenize and rejoin into a string

df1['clean_genres'] = df1['genres'].apply(preprocess_genres)

# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Apply the vectorizer to the cleaned genres column
tfidf_matrix = vectorizer.fit_transform(df_origin['clean_genres'])

# Convert the TF-IDF matrix to a DataFrame for easier inspection
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out()+'_genre', index = df_origin.index)

# Display the resulting DataFrame of genre embeddings
df_combined = pd.concat([df1, vectorized_texts, tfidf_df], axis=1)
df_genre_only = pd.concat([df1, tfidf_df], axis=1)


In [890]:
df_genre_only


,lower_title,Title,URL,Script,Clean_Script,joy,neutral,sadness,surprise,anger,...,horror_genre,movie_genre,music_genre,mystery_genre,romance_genre,science_genre,thriller_genre,tv_genre,war_genre,western_genre
0,sombre (1998),Sombre (1998),/movie_script.php?movie=sombre,Thump him!\n Thump him!\n Behind...\n Behind y...,"[thump, him, thump, him, behind, behind, you, ...",0.200000,0.000000,0.266667,0.066667,0.200000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
1,savior (1998),Savior (1998),/movie_script.php?movie=savior,"Christian, how are you, son?\n Good. You misse...","[christian, how, are, you, son, good, you, mis...",0.000000,0.080000,0.280000,0.080000,0.240000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
2,"aloha, bobby and rose (1975)","Aloha, Bobby and Rose (1975)",/movie_script.php?movie=aloha-bobby-and-rose,"From the big\n band years of the '40s,\n Artie...","[from, the, big, band, years, of, the, 40s, ar...",0.181818,0.254545,0.036364,0.236364,0.090909,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
3,money train (1995),Money Train (1995),/movie_script.php?movie=money-train,MONEY TRAIN\n Merry Christmas.\n Watch it!\n T...,"[money, train, merry, christmas, watch, it, th...",0.074074,0.148148,0.092593,0.055556,0.277778,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4,blue story (2019),Blue Story (2019),/movie_script.php?movie=blue-story,1\n [RADIO CUTTING IN AND OUT]\n [MALE REPORTE...,"[radio, cutting, in, and, out, male, reporter,...",0.083333,0.159091,0.181818,0.075758,0.371212,...,0.0,0.0,0.838344,0.0,0.545142,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19619,marius (1931),Marius (1931),/movie_script.php?movie=marius,"- Hey, Marius.\n - Yes, Fanny?\n - What are yo...","[hey, marius, yes, fanny, what, are, you, thin...",0.076923,0.142857,0.252747,0.142857,0.131868,...,0.0,0.0,0.000000,0.0,0.697964,0.0,0.000000,0.0,0.0,0.0
19620,my brother the devil (2012),My Brother the Devil (2012),/movie_script.php?movie=my-brother-the-devil,Brother!\n What is it? Ah...\n - All fucked up...,"[brother, what, is, it, ah, all, fucked, up, m...",0.042553,0.106383,0.106383,0.234043,0.212766,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
19621,hellion (2014),Hellion (2014),/movie_script.php?movie=hellion,Jacob! Wait!\n Check this out guys!\n Hey! Com...,"[jacob, wait, check, this, out, guys, hey, com...",0.128205,0.102564,0.230769,0.153846,0.230769,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.814860,0.0,0.0,0.0
19622,cross wars (2017),Cross Wars (2017),/movie_script.php?movie=cross-wars,"Hi, you don't happen to have the...\n The cros...","[hi, you, dont, happen, to, have, the, the, cr...",0.093333,0.360000,0.146667,0.093333,0.040000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


In [809]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

In [810]:
df = df_genre_only
features = df.drop(columns=df.select_dtypes(include='object').columns)

# Standardize the features for better clustering
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [811]:
# import matplotlib.pyplot as plt

# inertia = []
# K_range = range(1, 100)

# for k in K_range:
#     kmeans = KMeans(n_clusters=k, random_state=100)
#     kmeans.fit(features_scaled)
#     inertia.append(kmeans.inertia_)

# # Plot the Elbow Method
# plt.figure(figsize=(8,5))
# plt.plot(K_range, inertia, marker='o')
# plt.xlabel('Number of Clusters (K)')
# plt.ylabel('Inertia')
# plt.title('Elbow Method for Optimal K')
# plt.show()

In [812]:
kmeans = KMeans(n_clusters=200, random_state=42)
df['Cluster'] = kmeans.fit_predict(features_scaled)

# Show sample clustered data
df[['Title', 'sadness', 'joy', 'Cluster', 'genres']].head()

,Title,sadness,joy,Cluster,genres
0,Sombre (1998),0.266667,0.200000,136,DRAMA HORROR
1,Savior (1998),0.280000,0.000000,7,HISTORY DRAMA WAR
2,"Aloha, Bobby and Rose (1975)",0.036364,0.181818,103,DRAMA ROMANCE ACTION
3,Money Train (1995),0.092593,0.074074,51,ACTION COMEDY CRIME
4,Blue Story (2019),0.181818,0.083333,124,CRIME DRAMA ROMANCE


In [884]:
df[df['Cluster']==20]

,lower_title,Title,URL,Script,Clean_Script,joy,neutral,sadness,surprise,anger,...,movie_genre,music_genre,mystery_genre,romance_genre,science_genre,thriller_genre,tv_genre,war_genre,western_genre,Cluster
34,the addams family (1991),The Addams Family (1991),/movie_script.php?movie=the-addams-family,"Think of it, Thing.\n For 25 years,\n we've at...","[think, of, it, thing, for, years, weve, attem...",0.101695,0.033898,0.322034,0.271186,0.050847,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
46,youth (2015),Youth (2015),/movie_script.php?movie=youth,"1\n You can't smoke here.\n What, even outside...","[you, cant, smoke, here, what, even, outside, ...",0.111111,0.083333,0.277778,0.236111,0.138889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
132,"good morning, miss dove (1955)","Good Morning, Miss Dove (1955)",/movie_script.php?movie=good-morning-miss-dove,"'This is Liberty Hill, a small town,\n ..where...","[this, is, liberty, hill, a, small, town, wher...",0.224719,0.168539,0.359551,0.067416,0.033708,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
169,mishima: a life in four chapters (1985),Mishima: A Life in Four Chapters (1985),/movie_script.php?movie=mishima-a-life-in-four...,Yukio Mishima was\n Japan's most celebrated au...,"[yukio, mishima, was, japans, most, celebrated...",0.134615,0.096154,0.250000,0.134615,0.096154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
571,center stage: turn it up (2008),Center Stage: Turn It Up (2008),/movie_script.php?movie=center-stage-turn-it-up,1\n (PEOPLE CHATTERING)\n All up in the cipher...,"[people, chattering, all, up, in, the, cipher,...",0.378378,0.283784,0.121622,0.135135,0.013514,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18940,midnight son (2011),Midnight Son (2011),/movie_script.php?movie=midnight-son,Deep breath.\n Another.\n I need you\n to be h...,"[deep, breath, another, i, need, you, to, be, ...",0.000000,0.153846,0.153846,0.192308,0.269231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
19010,white fang (1991),White Fang (1991),/movie_script.php?movie=white-fang,[Indistinct Voices]\n Give it some slack.\n [G...,"[indistinct, voices, give, it, some, slack, gr...",0.116279,0.186047,0.069767,0.186047,0.139535,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
19212,blondie of the follies (1932),Blondie of the Follies (1932),/movie_script.php?movie=blondie-of-the-follies,But I have that date.\n I'm not going to let y...,"[but, i, have, that, date, im, not, going, to,...",0.210000,0.210000,0.170000,0.160000,0.050000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
19358,cat ballou (1965),Cat Ballou (1965),/movie_script.php?movie=cat-ballou,"Well, now friends just lend an ear For you're ...","[well, now, friends, just, lend, an, ear, for,...",0.032258,0.064516,0.096774,0.080645,0.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20


In [808]:
film_string = 'Django Unchained (2012)' # Title (Year)


overview = df_combined[df_combined['Title']==film_string][['Title','Cluster']]
print(overview)
cluster = int(overview['Cluster'])
df_combined[df_combined['Cluster']==cluster][['Title','words_per_minute', 'anger']]


KeyError: "['Cluster'] not in index"

In [943]:
from sklearn.cluster import DBSCAN

df = df_genre_only
features = df.drop(columns=['tmdbId', 'runtime']).drop(columns=df.select_dtypes(include='object'))


#Standardize the features
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)
features_scaled_df = pd.DataFrame(features_scaled)

# Apply DBSCAN
dbscan = DBSCAN(eps=0.35, min_samples=5)  # Adjust eps and min_samples as needed
df['Cluster'] = dbscan.fit_predict(features_scaled)
#features_scaled_df.describe().apply(lambda x: round(x,2))
print('Unclustered Films: ',len(df[df.Cluster == -1]))


Unclustered Films:  2248


In [944]:
df['Cluster'].value_counts().median()


12.0

In [946]:
#film_string = 'Grown Ups (2010)' # Title (Year)
film_string = 'Star Wars (1977)' # Title (Year)
#film_string = 'Django Unchained (2012)' # Title (Year)

overview = df[df['Title']==film_string][['Title','Cluster']]
print(overview)
cluster = int(overview['Cluster'])
df[df['Cluster']==cluster][['Title','sentiment_entropy','words_per_minute', 'genres']]

                  Title  Cluster
10309  Star Wars (1977)       51


/var/folders/xt/0g_4gstj4v7gqfpdt0wv97280000gn/T/ipykernel_6473/901272863.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cluster = int(overview['Cluster'])


,Title,sentiment_entropy,words_per_minute,genres
165,Hyperland (2021),1.364902,64.412844,SCIENCE FICTION
332,The Cellar (2022),1.176691,45.510638,HORROR MYSTERY
595,Baked in Brooklyn (2016),1.322270,97.72093,COMEDY CRIME DRAMA
937,Cheap Thrills (2013),1.322270,55.488636,COMEDY CRIME DRAMA
1541,Inside the Edge: A Professional Blackjack Adve...,1.322270,149.361111,DOCUMENTARY
...,...,...,...,...
18919,Last Man Standing (1996),1.322270,73.19802,CRIME ACTION DRAMA
19079,Dream House (2011),1.440645,59.52381,MYSTERY DRAMA THRILLER
19176,Drunk Wedding (2015),1.322270,114.049383,COMEDY
19356,1917 (2019),1.322270,42.596639,WAR HISTORY THRILLER


In [925]:
df[df['Cluster']==17][['Title','joy', 'neutral', 'sentiment_entropy','words_per_minute', 'genres']]

,Title,joy,neutral,sentiment_entropy,words_per_minute,genres
31,Nick Kroll: Little Big Boy (2022),0.223684,0.157895,1.322270,134.133333,COMEDY
134,Louis Theroux: Behind Bars (2008),0.036585,0.304878,1.322270,142.683333,DOCUMENTARY TV MOVIE
147,A Fantastic Woman (2017),0.125000,0.145833,1.322270,44.971154,DRAMA
732,All or Nothing (2002),0.048780,0.121951,1.322270,64.21875,DRAMA
800,Operation Mekong (2016),0.067797,0.220339,1.322270,48.290323,ACTION ADVENTURE CRIME
...,...,...,...,...,...,...
18720,Guns of the Magnificent Seven (1969),0.021739,0.282609,1.322270,45.92381,WESTERN
18994,Rambo: Last Blood (2019),0.111111,0.000000,1.494785,11.247191,ACTION THRILLER DRAMA
19028,Every Which Way but Loose (1978),0.046154,0.215385,1.322270,62.472727,ACTION COMEDY ADVENTURE
19303,No Way Out (2022),0.073171,0.121951,1.322270,48.853933,THRILLER CRIME ACTION


In [917]:
df = df.drop(columns=['title'])

In [918]:
#Clusters in Pure Classes
df[df.comedy_genre == 1][['Title','genres','Cluster']].value_counts() #comedy all in one cluster
df[df.comedy_genre == 1]['Cluster'].value_counts() #comedy all in one cluster
#df[df.drama_genre == 1]['Cluster'].value_counts() #drama in 2 clusters
#df[df.action_genre == 1]['Cluster'].value_counts() #action in one cluster
#df[df.horror_genre == 1]['Cluster'].value_counts() #horror in one cluster
# df[df.animation_genre == 1]['Cluster'].value_counts() #animation in one cluster

Cluster
 14    1005
-1        6
Name: count, dtype: int64

## Log Scaling the Script



In [936]:

from sklearn.cluster import DBSCAN

df = df_genre_only
#df.set_index('tmdbId', drop = True, inplace = True)

features = df.drop(columns=['tmdbId', 'runtime']).drop(columns=df.select_dtypes(include='object'))
script_columns = ['joy', 'neutral', 'sadness', 'surprise', 'anger', 'disgust', 'fear',
       'sentiment_entropy', 'sentiment_std', 'hapax', 'fkgl', 'fre', 'smog',
       'fog', 'word count', 'TTR', 'script_length', 'mean_word_length',
       'words_per_minute']
features[script_columns] =  np.sqrt(features[script_columns])

# # Standardize the genre features
scaler = MinMaxScaler()

features_scaled = scaler.fit_transform(features)
features_scaled_df = pd.DataFrame(features_scaled)
features_scaled_df.columns = features.columns

# Apply DBSCAN
dbscan = DBSCAN(eps=0.3, min_samples=5)  # Adjust eps and min_samples as needed
df['Cluster'] = dbscan.fit_predict(features_scaled_df)
pd.DataFrame(features_scaled_df).describe().apply(lambda x: round(x,2))
print('Unclustered Films: ',len(df[df.Cluster == -1]))

# features[features.isna().any(axis=1)][['joy', 'neutral', 'sadness', 'surprise', 'anger', 'disgust', 'fear',
#        'sentiment_entropy', 'sentiment_std', 'hapax', 'fkgl', 'fre', 'smog',
#        'fog', 'word count', 'TTR', 'script_length', 'mean_word_length',
#        'words_per_minute']
# ]

Unclustered Films:  5910


In [941]:
#film_string = 'Grown Ups (2010)' # Title (Year)
film_string = 'Star Wars (1977)' # Title (Year)
#film_string = 'Django Unchained (2012)' # Title (Year)
#film_string = 'Wedding Crashers (2005)' # Title (Year)
overview = df[df['Title']==film_string][['Title','Cluster']]
print(overview)
cluster = int(overview['Cluster'])
my_cluster = df[df['Cluster']==cluster][['Title','sentiment_entropy','words_per_minute', 'genres']]
my_cluster.genres.value_counts()

                  Title  Cluster
10309  Star Wars (1977)       29


/var/folders/xt/0g_4gstj4v7gqfpdt0wv97280000gn/T/ipykernel_6473/2385971298.py:7: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  cluster = int(overview['Cluster'])


genres
DRAMA                              12
COMEDY                              8
HORROR                              4
COMEDY DRAMA                        3
DRAMA ROMANCE                       3
                                   ..
SCIENCE FICTION THRILLER HORROR     1
ANIMATION COMEDY FAMILY             1
MYSTERY THRILLER DRAMA              1
HORROR THRILLER DRAMA               1
THRILLER DRAMA MYSTERY              1
Name: count, Length: 94, dtype: int64

In [866]:
# # BigQuery
# from google.cloud import bigquery, bigquery_storage
# bigquery_client = bigquery.Client(project='film-wizard-453315')

# proccessed_df = df

# project_id = 'film-wizard-453315'  # GCP Project ID
# dataset_id = "clustered_movies"  # dataset name

# table_id = "dbscan_all"  # Replace with your table name

# client = bigquery_client
# proccessed_df.to_gbq(f"{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

## No Genres



In [786]:
#merge dataframes 
df_script = pd.merge(script_df, tmdb_df, how = 'inner', on = 'lower_title')
df_script[['fear', 'sadness', 'joy', 'neutral', 'anger', 'surprise', 'disgust']] = df_script[['fear', 'sadness', 'joy', 'neutral', 'anger', 'surprise', 'disgust']].fillna(0)
df_script[['sentiment_entropy', 'sentiment_std']] = df_script[['sentiment_entropy', 'sentiment_std']].fillna(df_script[['sentiment_entropy', 'sentiment_std']].mean())
df_script = df_script.dropna()
df_script = df_script.drop_duplicates(subset=['lower_title'])
df_script['words_per_minute'] = df['script_length'] / df['runtime']
df_script.set_index('tmdbId', drop = True, inplace = True)
df_script2 = df_script.drop(columns=['word count','runtime','release_year','genres', 'title'])
df_script2

features2 = df_script2.drop(columns=df_script2.select_dtypes(include='object').columns)

# Standardize the features
scaler = MinMaxScaler()
features2_scaled = scaler.fit_transform(features2)
features2_scaled_df = pd.DataFrame(features2_scaled)

# Apply DBSCAN
dbscan = DBSCAN(eps=0.1, min_samples=5)  # Adjust eps and min_samples as needed
df_script['Cluster'] = dbscan.fit_predict(features2_scaled)
#features_scaled_df.describe().apply(lambda x: round(x,2))
print('Unclustered Films: ',len(df_script[df_script.Cluster == -1]))



Unclustered Films:  12425


In [481]:
df_script

,lower_title,Title,URL,Script,Clean_Script,joy,neutral,sadness,surprise,anger,...,word count,TTR,script_length,mean_word_length,title,runtime,release_year,genres,words_per_minute,Cluster
tmdbId,,,,,,,,,,,,,,,,,,,,,
72091,voyage to the prehistoric planet (1965),Voyage to the Prehistoric Planet (1965),/movie_script.php?movie=voyage-to-the-prehisto...,"The year, 2020. The moon has been\n explored a...","[the, year, the, moon, has, been, explored, an...",0.080000,0.220000,0.080000,0.220000,0.020000,...,1128,0.226188,4987,4.207740,Voyage to the Prehistoric Planet,78.0,1965,SCIENCE FICTION ADVENTURE,63.935897,-1
569499,detainment (2018),Detainment (2018),/movie_script.php?movie=detainment,1\n Jon.\n Do you know why we\n wanna talk to ...,"[jon, do, you, know, why, we, wanna, talk, to,...",0.000000,0.250000,0.035714,0.071429,0.000000,...,488,0.160158,3047,3.692484,Detainment,30.0,2018,DRAMA CRIME DOCUMENTARY,101.566667,-1
833339,speak no evil (2022),Speak No Evil (2022),/movie_script.php?movie=speak-no-evil-2022,[car traveling]\n - [suspension squeaking]\n -...,"[car, traveling, suspension, squeaking, wind, ...",0.306122,0.183673,0.163265,0.142857,0.020408,...,954,0.211483,4511,4.444912,Speak No Evil,97.0,2022,HORROR THRILLER DRAMA,46.505155,0
44751,dossier k. (2009),Dossier K. (2009),/movie_script.php?movie=dossier-k,BLood is never unavenged\n (Art. 917 of the Ka...,"[blood, is, never, unavenged, art, of, the, ka...",0.015625,0.218750,0.078125,0.109375,0.203125,...,1358,0.212420,6393,4.168466,Dossier K.,120.0,2009,ACTION CRIME DRAMA,53.275,-1
42202,cyclone (1978),Cyclone (1978),/movie_script.php?movie=cyclone,(suspenseful synthwave music)\n - Now ladies a...,"[suspenseful, synthwave, music, now, ladies, a...",0.137255,0.078431,0.156863,0.039216,0.019608,...,1006,0.207166,4856,4.285420,Cyclone,118.0,1978,HORROR,41.152542,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11169,spartan (2004),Spartan (2004),/movie_script.php?movie=spartan,You had your whole life\n to prepare for this ...,"[you, had, your, whole, life, to, prepare, for...",0.037500,0.250000,0.025000,0.137500,0.112500,...,1482,0.177040,8371,3.856290,Spartan,102.0,2004,MYSTERY ACTION DRAMA,82.068627,0
48918,shadow of the wolf (1992),Shadow of the Wolf (1992),/movie_script.php?movie=shadow-of-the-wolf,AGAGUK\n (aka Shadow of the Wolf)\n GRAND NORD...,"[agaguk, aka, shadow, of, the, wolf, grand, no...",0.033333,0.033333,0.166667,0.066667,0.400000,...,622,0.190681,3262,3.693440,Shadow of the Wolf,112.0,1992,ADVENTURE,29.125,-1
13184,bangkok dangerous (2008),Bangkok Dangerous (2008),/movie_script.php?movie=bangkok-dangerous,My job takes me to a lot of places.\n It's got...,"[my, job, takes, me, to, a, lot, of, places, i...",0.100000,0.150000,0.150000,0.100000,0.200000,...,596,0.296076,2013,3.884252,Bangkok Dangerous,99.0,2008,ACTION CRIME THRILLER,20.333333,-1


In [569]:
df_script[df_script['Cluster']==1][['Title','sentiment_entropy','words_per_minute', 'genres']]
def script_classifier(sent_ent):
    if sent_ent > 1.4:
        return 2
    elif sent_ent > 1:
        return 1
    else:
        return 0 
df_script['sent_ent_class'] = df_script['sentiment_entropy'].apply(script_classifier)
df_script['sent_ent_class'].value_counts()


sent_ent_class
1    9557
2    8416
0    1651
Name: count, dtype: int64

In [570]:
df_script['Cluster'].value_counts()

Cluster
-1     12425
 0      6724
 51       19
 14       18
 7        15
       ...  
 40        2
 79        2
 66        2
 8         2
 73        1
Name: count, Length: 86, dtype: int64

In [926]:
df[['tmdbId','Cluster']]

,tmdbId,Cluster
0,56653,-1
1,2042,0
2,47178,281
3,11517,1
4,621191,259
...,...,...
19619,42818,36
19620,84316,5
19621,244536,10
19622,412452,59


In [927]:
proccessed_df = df[['tmdbId','Cluster']]

project_id = 'film-wizard-453315'  # GCP Project ID
dataset_id = "clustered_movies"  # dataset name

table_id = "dbscan_v2_slim"  # Replace with your table name

client = bigquery_client
proccessed_df.to_gbq(f"{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")

/var/folders/xt/0g_4gstj4v7gqfpdt0wv97280000gn/T/ipykernel_6473/3013101614.py:9: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  proccessed_df.to_gbq(f"{dataset_id}.{table_id}", project_id=project_id, if_exists="replace")
100%|███████████████████████████████████████████████████████| 1/1 [00:00<00:00, 6887.20it/s]
